1. import necessary library

In [1]:
import pandas as pd
import dash
from dash import html, dcc, Input, Output
import plotly.graph_objs as go

2. creat sample dataframe for testing, will replace by our dataset later

In [27]:
# Define data as a DataFrame
data = pd.DataFrame({
    'symptom': [
        'Cough', 'Chest Pain', 'Short Breath', 'Headache', 'Fever', 'Abdominal Pain', 'Dizziness', 
        'Abdominal Pain', 'Vomiting', 'Missed Period', 'Swelling', 'Chest Pain'
    ],
    'condition': [
        'Common Cold', 'Heart Attack', 'Asthma', 'Migraine', 'Influenza', 'Appendicitis', 'Low Blood Pressure',
        'Gallstones', 'Food Poisoning', 'Pregnancy', 'Heart Failure', 'Anxiety'
    ],
    'department': [
        'General Medicine', 'Cardiology', 'Pulmonology', 'Neurology', 'General Medicine', 'Surgical', 'Cardiology',
        'Surgical', 'Gastroenterology', 'Obstetrics', 'Cardiology', 'Psychiatry'
    ],
    'age_range': [
        'Any', 'Adults', 'Any', 'Any', 'Any', 'Adults', 'Adults',
        'Adults', 'Any', 'Adults', 'Elderly', 'Adults'
    ],
    'gender': [
        'Any', 'Any', 'Any', 'Any', 'Any', 'Any', 'Any',
        'Any', 'Any', 'Female', 'Any', 'Any'
    ],
    'nature': [
        'Acute', 'Acute', 'Chronic', 'Chronic', 'Acute', 'Acute', 'Chronic',
        'Chronic', 'Acute', 'Chronic', 'Chronic', 'Acute'
    ],
     'confidence_interval': [
        '95-99%', '90-95%', '85-90%', '80-85%', '75-80%', '95-99%', '90-95%',
        '85-90%', '80-85%', '95-99%', '75-80%', '70-75%'
    ]
})


In [29]:
# Create a Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Health Self-Check Interactive Platform", style={'color': '#007BFF'}),
    dcc.Dropdown(
        id='nature-dropdown',
        options=[
            {'label': 'Chronic', 'value': 'Chronic'},
            {'label': 'Acute', 'value': 'Acute'}
        ],
        placeholder="Select Problem Nature",
        style={'width': '200px'}
    ),
    dcc.Dropdown(
        id='age-group-dropdown',
        options=[
            {'label': 'Children (0-12)', 'value': 'Children'},
            {'label': 'Teenagers (13-19)', 'value': 'Teenagers'},
            {'label': 'Adults (20-64)', 'value': 'Adults'},
            {'label': 'Seniors (65+)', 'value': 'Seniors'}
        ],
        placeholder="Select Age Group",
        style={'margin-right': '10px', 'width': '200px'}
    ),
    dcc.Dropdown(
        id='gender-dropdown',
        options=[
            {'label': 'Male', 'value': 'Male'},
            {'label': 'Female', 'value': 'Female'},
            {'label': 'Diversity', 'value': 'Diversity'}
        ],
        placeholder="Select Gender",
        style={'width': '200px', 'margin-right': '10px'}
    ),
    dcc.Dropdown(
        id='symptom-dropdown',
        options=[{'label': s, 'value': s} for s in data['symptom'].unique()],
        placeholder="Select a symptom",
        style={'width': '50%', 'margin-top': '0px', 'margin-bottom': '10px'}
    ),
    html.Button('Submit', id='submit-button', n_clicks=0, style={'margin-bottom': '10px',  'background-color': '#28A745', 'color': 'white'}),
    html.Div(id='output-container',style={'margin-top': '20px', 'color': '#17A2B8', 'background-color': 'white', 'padding': '10px', 'border-radius': '5px', 'border': '2px solid #dee2e6'})
])

@app.callback(
    Output('output-container', 'children'),
    [Input('submit-button', 'n_clicks')],
    [dash.dependencies.State('nature-dropdown','value'),
     dash.dependencies.State('age-group-dropdown', 'value'),
     dash.dependencies.State('gender-dropdown', 'value'),
     dash.dependencies.State('symptom-dropdown', 'value')
     ]
)
def update_output(n_clicks, nature, age, gender, symptom):
    if n_clicks > 0:
        if not all([age, nature, gender, symptom]):
            return html.Div([
                html.H3("Please fill out all fields.", style={'color': 'red'}),
            ])
        result = data[data['symptom'].str.lower() == symptom.lower()]
        if not result.empty:
           condition = result['condition'].values[0]
           department = result['department'].values[0]
           confidence = result['confidence_interval'].values[0]
           return html.Div([
                html.H3(f"Results for the input:", style={'color': 'green'}),
                html.P(f"It is a {nature} problem."),
                html.P(f"Symptom: {symptom}"),
                html.P(f"Age: {age}"),
                html.P(f"Gender: {gender}"),
                html.P(f"Possible Condition: {condition}", style={'font-weight': 'bold'}),
                html.P(f"Recommended Department: {department}", style={'background-color': '#FFC107', 'color': 'black'}),
                html.P(f"Confidence Interval: {confidence}", style={'color': '#28A745'}),
            ], style={'width': '50%','border': '2px solid green', 'padding': '10px', 'border-radius': '5px','background-color': 'white'})
        else:
            return html.Div([
                html.H3("No results found for the selected symptom.", style={'color': 'red'})
            ])
    return html.Div()


if __name__ == '__main__':
    app.run_server(debug=True)
